<a href="https://colab.research.google.com/github/Julian-Banks/EEE4022S_BNKJUL001_Thesis/blob/main/PythonWorkspace/EMSv0_A2C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gymnasium
!pip install stable_baselines3[extra]
%load_ext tensorboard

In [1]:
! git clone https://github.com/Julian-Banks/EEE4022S_BNKJUL001_Thesis


Cloning into 'EEE4022S_BNKJUL001_Thesis'...
remote: Enumerating objects: 411, done.
remote: Counting objects: 100% (121/121), done.
remote: Compressing objects: 100% (101/101), done.
remote: Total 411 (delta 31), reused 103 (delta 17), pack-reused 290
Receiving objects: 100% (411/411), 141.49 MiB | 28.70 MiB/s, done.
Resolving deltas: 100% (97/97), done.
Updating files: 100% (289/289), done.


This is very much a first working model and very simple
First major problem is that it only has one load! no AC DC vibe
also has no converter
All of the loads and forecasts are random rn but I need to load in the real files.
only one form of generation, no solar/wind difference

Tech challenges:
  If the data is normalised, how do I calculate the actual power used, in the batery exct, unless they are all normalised using the same metric,
  could save thier normalisation values, un-normalise them to perform calculations and then renormalise them?
  

In [10]:
import gymnasium as gym
import numpy as np
from gymnasium import spaces
import datetime
from stable_baselines3 import PPO, A2C, DQN
from google.colab import drive
import os

class EMSv0(gym.Env):
    """Custom Environment that follows gym interface."""

    metadata = {"render_modes": ["human"], "render_fps": 30}

    def __init__(self,bat_threshold = 0.1,purchase_price = [1,1,1,1,1,1,1,1,2,2,2,2] , episode_len = 8760,num_preds = 24,render_mode = "none"):

        super(EMSv0, self).__init__()

        #define time frame
        self.current_step = 0
        self.final_step = int(episode_len) #one years worth of steps

        #Might make a function for these
        #fill all of the actual loads NB!!! is just random for now NB!!! is normalised 0-1
        self.actual_load = np.random.rand(self.final_step+num_preds+1).astype(np.float32) #will load from a file or something
        #fill all of the actual generation steps.
        self.actual_gen   = np.random.rand(self.final_step+num_preds+1).astype(np.float32) #will load from file or something

        #define the purchase price for every step of the year
        purchase_price = np.array(purchase_price).astype(np.float32)
        repetitions    = (self.final_step+num_preds+1) // len(purchase_price)
        remainder      = (self.final_step+num_preds+1) % len(purchase_price)
        self.purchase_price =np.concatenate([purchase_price]*repetitions+[purchase_price[:remainder]])#need to read in from somewhere

        #define var for storing the excess gen
        self.excess_gen = 0
        #define a var for determine amount purchased per step (dont want to make it total as this will incure growing penalties for the Agent if used in reward structure)
        self.step_purchased = 0
        #define the battery low threshold
        self.bat_threshold = np.float32(bat_threshold)
        #define default action
        self.default_action = 0
        #define actions and observations space
        n_actions = 2 # keeping it simple

        self.num_preds = num_preds # day ahead predictions
        self.action_space = spaces.Discrete(n_actions)
        # Dict space to store all the different things
        self.observation_space = spaces.Dict({
                "load_forecast": gym.spaces.Box(low=0, high=np.inf, shape=(1,num_preds), dtype=np.float32),
                "gen_forecast": gym.spaces.Box(low=0, high=np.inf, shape=(1,num_preds), dtype=np.float32),
                "price_forecast": gym.spaces.Box(low=0, high=np.inf, shape=(1,num_preds+1), dtype=np.float32),
                "bat_level": gym.spaces.Box(low=0, high=np.inf, shape=(1,), dtype=np.float32),
                "current_load": gym.spaces.Box(low=0, high=np.inf, shape=(1,), dtype=np.float32),
                "current_gen": gym.spaces.Box(low=0, high=np.inf, shape=(1,), dtype=np.float32),
                 })

    def step(self, action):

        #update the current state with the action (needs to be done before current_step is inc since we want to apply the action to the previous step to get the current state)
        self.update_state(action)
        #Calculate reward from the action
        reward = self.calc_reward()

        #inc time step into Future
        self.current_step += 1
        #get next observation (for next time step)
        observation = self.get_obs()
        #Set terminated to False since there are no failure states
        self.terminated = False
        #Check if timelimit reached
        self.truncated = False if self.current_step<self.final_step else True
        #dont know what to put into info for now
        info = {}
        return observation, reward, self.terminated, self.truncated, info

    def reset(self, seed=None, options=None):
        super().reset(seed = seed, options=options)

        self.current_step = 0
        self.terminated = False
        self.truncated = False

        #reset these cause I dont want the model to just memorise the random data (so it gets changed every reset)
        #fill all of the actual loads NB!!! is just random for now NB!!! is normalised 0-1
        self.actual_load = np.random.rand(self.final_step+self.num_preds+1).astype(np.float32) #will load from a file or something
        #fill all of the actual generation steps.
        self.actual_gen   = np.random.rand(self.final_step+self.num_preds+1).astype(np.float32) #will load from file or something


        #reset the state
        self.battery_level = 0.5
        self.current_load = self.actual_load[0]
        self.excess_gen = 0
        self.step_purchased = 0
        #get the first observation
        observation = self.get_obs()
        #Still don't know what to do with info
        info = {}
        return observation, info

    def render(self):
        #Reaaaaalllyyyy want to render something, Maybe the curent load as a point, the forecasts as a plot and the bat levels as bar
        pass

    def close(self):
        #don't think i need this for my application
        pass

    def update_state(self, action):
        #Update current state with actions
        if action == 0: #do nothing action
            self.standby()
        elif action == 1: #buy from Grid
            self.purchase()
        else:  #error case
            raise ValueError(
              f"Received invalid action = {action} which is not part of the action space."
            )
        #case list for each action?

    def calc_reward(self):
        #Calculate reward based on the state
        reward = -self.step_purchased*self.purchase_price[self.current_step]

        return reward

    def get_obs(self):
        #Fill the observation space with the next observation

        #Get Forecasts Will probaly write a function for this? idk maybe a schlep to return all the info
        load_forecast  = np.array( [self.actual_load[self.current_step+1: self.current_step + self.num_preds+1]] , dtype = np.float32) #will load from a file or something
        if load_forecast.shape != (1,24):
            print(f"load_forecast shape is {load_forecast.shape}. Current step is {self.current_step}")
        gen_forecast   = np.array( [self.actual_gen[self.current_step+1: self.current_step + self.num_preds+1]] , dtype = np.float32) #will load from a file or something
        if gen_forecast.shape != (1,24):
            print(f"gen_forecast shape is {gen_forecast.shape}. Current step is {self.current_step}")
        #get the prices for the current frame and the next 24 hours. Maybe will cut this down since that seems like a lot of info
        price_forecast = np.array( [self.purchase_price[self.current_step:self.current_step+self.num_preds+1]] , dtype = np.float32)
        #Just for readibility of the dict object
        bat_level      = np.array([self.battery_level] , dtype= np.float32)
        current_load   = np.array([self.actual_load[self.current_step]], dtype = np.float32)
        current_gen    = np.array([self.actual_gen[self.current_step]], dtype  = np.float32)


        obs = dict({
                "bat_level":      bat_level,
                "current_gen" :   current_gen,
                "current_load":   current_load,
                "gen_forecast":   gen_forecast,
                "load_forecast":  load_forecast,
                "price_forecast": price_forecast,
        })
        return obs

    def standby(self):
        #ems stands by, load is met by generation, battery and then grid
        #if there is excess generation it is used to charge the batteries

        #define step_gen and step_load for readability
        step_gen  =  self.actual_gen[self.current_step]
        step_load =  self.actual_load[self.current_step]
        battery   =  self.battery_level
        #check for gen meeting load
        if step_load <= step_gen :
            #set the purchased elect to 0 since gen meets load
            self.step_purchased = 0
            #calulate the excess elec that was generated
            step_excess = step_gen - step_load
            #check if battery needs to be charged
            if battery < 1 :
                #check if the excess amount that was generated is less than the available capacity
                if 1-battery-step_excess > 0:
                    self.battery_level += step_excess
                else:
                    #if the excess is greater than the availability then charge till full
                    self.battery_level = 1
                    #set step excess to excess minus the amount used to charge
                    step_excess -= (1-battery)
                    self.excess_gen += step_excess
            else:
                #if the battery is full then just inc excess_gen
                self.excess_gen += step_excess
        else:
            #if the generation does not meet load
            step_shortfall = step_load - step_gen
            #checking if battery is above a threshold.
            if battery > self.bat_threshold:
                #check if battery has enough capacity to meet the load
                if battery - step_shortfall >= self.bat_threshold:
                    #if it does then subtract the shortfall from battery level
                    self.battery_level -= step_shortfall
                    #set the purchased variable to 0 since nothing was purchased
                    self.step_purchased = 0
                else:
                    #set the battery to min value and purchase the rest from the grid
                    self.battery_level = self.bat_threshold
                    #calculate how much needs to be purchased
                    step_shortfall -= (battery - self.bat_threshold)
                    self.step_purchased = step_shortfall
            else:
                #no battery available, therefore everything needs to be bought from the grid.
                self.step_purchased = step_shortfall

    def purchase(self):
        #purchase electricity to charge battery even if there is enough generation (I assume this will be used to buy at lower prices)
        #get values for readability
        step_load = self.actual_load[self.current_step]
        step_gen  = self.actual_gen[self.current_step]
        battery = self.battery_level

        #calculate the total power need (the load plus the amount that the battery needs to charge)
        total_need = step_load + (1-battery)
        #if the generation is less than the need then purchase the remainder
        if step_gen<total_need:
            #purchashing the shortfall
            self.step_purchased = total_need - step_gen
            #setting the battery levels to full
            self.battery_level = 1
        else:
            #if the gen is enough then set purchase to 0
            self.step_purchased  = 0
            #set the battery to fully charged
            self.battery_level = 1
            #inc excess_gen by caluclating the excess between the step gen and the total need (includes amount needed to charge the battery)
            self.excess_gen += (step_gen - total_need)


Check the environment with stable_baselines3 check_env.

In [11]:
from stable_baselines3.common.env_checker import check_env
env = EMSv0()
check_env(env,warn = True)

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/env_checker.py:244: UserWarning: Your observation 
gen_forecast has an unconventional shape (neither an image, nor a 1D vector). We recommend you to flatten the 
observation to have only a 1D vector or use a custom policy to properly process the data.
  warnings.warn(

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/env_checker.py:244: UserWarning: Your observation 
load_forecast has an unconventional shape (neither an image, nor a 1D vector). We recommend you to flatten the 
observation to have only a 1D vector or use a custom policy to properly process the data.
  warnings.warn(

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/env_checker.py:244: UserWarning: Your observation 
price_forecast has an unconventional shape (neither an image, nor a 1D vector). We recommend you to flatten the 
observation to have only a 1D vector or use a custom policy to properly process the data.
  warnings.warn(

Evaluate the base model (no EMS, just using standby mode)

In [12]:
#define the base environment
base_env = EMSv0(episode_len = 7*24)
#reset the environment and save the obs
#going to run it 100 times to get a benchmark
#reset score
score = 0
for step in range(1000):
    obs,_    = base_env.reset()
    #ensure that the exit condition is reset
    truncated = False
    #define the action to take
    action_standby = 0

    while not truncated:
        obs,reward,terminated,truncated,info = base_env.step(action_standby)
        score += reward

print(f"Done iteration! Total reward accumulated is: {score/step}")

Done iteration! Total reward accumulated is: -14.025860077784014

Train a model

In [18]:
#create a new environment to train the model in.
train_env = EMSv0(episode_len = 7*24)

#mount the drive
drive.mount('/content/drive')
#define paths to logs and model saves
model_dir = "/content/drive/MyDrive/Colab Notebooks/EMSv0/models/A2C/"
log_dir   = "/content/drive/MyDrive/Colab Notebooks/EMSv0/models/A2Clogs/"

#make the appropriate directory if it does not exist
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
if not os.path.exists(log_dir):
    os.makedirs(log_dir)




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", 
force_remount=True).

**LOAD OR MAKE MODEL HERE!**

In [20]:

#Create the model with the MultiInputPolicy, use the training env, verbose is off because tensorboard loging is enabled
#model = A2C("MultiInputPolicy",train_env, verbose = 0, tensorboard_log = log_dir)

#Load model, fetch the latest (or whichever one you want from the model_dir)
#model_load = "/content/drive/MyDrive/Colab Notebooks/EMSv0/models/A2C/EMSv0_A2C20231004-0826.zip"
#model  = A2C.load(model_load, env = train_env)



In [ ]:
%tensorboard --logdir "/content/drive/MyDrive/Colab Notebooks/EMSv0/models/A2Clogs/"

while True:
    #define the name for the specific log
    log_name = "EMSv0_A2C"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    #make the model learn, set the reset to false so that it keeps its old learning
    model.learn(total_timesteps= 100000, tb_log_name = log_name,reset_num_timesteps=False)
    model.save(f"{model_dir}{log_name}")
    #open tensorboard

Launching TensorBoard...

In [ ]:
#define a test environment
test_env = EMSv0
#reset the environment and save the obs
obs,_    = base_env.reset()
#ensure that the exit condition is reset
truncated = False
#define the action to take
action_standby = 0
#reset score
score = 0
while not truncated:
    #predict the best next action
    action,_ = model.predict(obs)
    #step the model with the action
    obs,reward,terminated,truncated,info = base_env.step(action)
    #accumulate the score
    score += reward
print(f"Done! Total reward accumulated is: {score}")